In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import random
from math import exp
import xgboost as xgb
from nltk.stem import PorterStemmer
import re
#import distance

random.seed(321)
np.random.seed(321)

X_train = pd.read_json("../input/train.json")
X_test = pd.read_json("../input/test.json")


interest_level_map = {'low': 0, 'medium': 1, 'high': 2}
X_train['interest_level'] = X_train['interest_level'].apply(lambda x: interest_level_map[x])
X_test['interest_level'] = -1

#remove outliers
ulimit = np.percentile(X_train.price.values, 99)
X_train.ix[X_train['price']>ulimit,'price'] = ulimit

In [ ]:
feature_transform = CountVectorizer(stop_words='english', max_features=150)
X_train['features'] = X_train["features"].apply(lambda x: " ".join(["_".join(i.lower().split(" ")) for i in x]))
X_test['features'] = X_test["features"].apply(lambda x: " ".join(["_".join(i.lower().split(" ")) for i in x]))
#feature_transform.fit(list(X_train['features']) + list(X_test['features']))

In [ ]:
X_train.features.count()

In [ ]:
X_train['features'] = X_train['features'].apply(lambda x: ' '.join(['_'.join(i.split(' ')) for i in x]))
textcv = CountVectorizer(stop_words='english', max_features=200)
text_features = pd.DataFrame(textcv.fit_transform(X_train['features']).toarray(),
                                 columns=['f_' + format(x, '03d') for x in range(1, 201)],
                                 index=X_train.index)
X_train = pd.concat(objs=(X_train, text_features), axis=1)
#data_df.drop('features', axis=1, inplace=True)

In [ ]:
text_features.head()
X_train.head()